![図1.2 Sparkの紹介とインストール](images/PysparkSQL.png)

# 目次
1. 前のセクションの振り返りとSparkSQLの紹介
2. SparkSQLを用いてデータエンジニアリング一連の流れを再現
3. テーブル定義はどこに保存されている？
4. SparkSQLの他のTips(Insert文)
5. SparkSQLの他のTips(Hint文)
6. Update/Delete文？
7. SparkSQLを使うメリット、Dataframeを使うメリット


# 前回のセクションいかがでしたか？
前回のセクションでは、DataFrameを用いてデータの変換(ETLにおけるTの処理)を行いました。

しかし、ちょっとdataframeの操作は慣れてなくて。。。
という人多かったんじゃないでしょうか

```
df_after_t=df.where(df.gengo == "平成").groupBy("kenmei") \
    .agg(sf.avg("jinko_male").alias("male_avg"),sf.avg("jinko_female").alias("female_avg")) \
      .filter(df.kenmei != "人口集中地区以外の地区").sort("male_avg")
```

今回はデータをSQLで操作可能なSparkSQLを使いながら前回のチャプターと同じことをやっていこうと思います



# SparkSQLとは？
Apache HiveベースのSQLを実行できます(Mysqlの使い勝手に似ています。そのため、Mysqlの感覚で使ってみて使えないところをピンポイントで検索するといいと思います。)。  
データフレームの操作が苦手な人でも、SQLを知っていればSparkで読み込んだデータを操作してT（変換処理）が可能になります。

# Sparkを用いたバッチにおけるデータエンジニアリング一連の流れ(復習)

1. データソースの読み込み(今回は、人口統計データ(/dataset/jinkou.csv))　ETL(Extract Transform Load)で言うEの部分
2. 変換を行う(集計等を行う)　DataFrame処理 or SQL処理の２パターンで実行可能 ETL(Extract Transform Load)で言うTの部分
3. カラムナーフォーマットへ変換する ETL(Extract Transform Load)で言うTの部分
4. 出力したデータをみんなに見やすくするため(BIツールから参照できるように)テーブルを作成する ETL(Extract Transform Load)で言うLの部分

よくある、関数の羅列をするのではなく、実業務に沿った形で流れを紹介していきます。

In [2]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は次のチャプターで説明を行います。
spark = SparkSession.builder \
    .appName("chapter2") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()

21/10/12 11:00:14 WARN Utils: Your hostname, saitouyuuki-no-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.10 instead (on interface en0)
21/10/12 11:00:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/12 11:00:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# データソースの読み込み

In [4]:
from pyspark.sql.types import LongType, StructType, StructField, StringType
from pyspark.sql.functions import col

# カラム名、型、デフォルト値で設定していきます
struct = StructType([
    StructField("code", StringType(), False),
    StructField("kenmei", StringType(), False),
    StructField("gengo", StringType(), False),
    StructField("wareki", StringType(), False),
    StructField("seireki", StringType(), False),
    StructField("chu", StringType(), False),
    StructField("sokei", StringType(), False),
    StructField("jinko_male", StringType(), False),
    StructField("jinko_female", StringType(), False)
])
df=spark.read.option("multiLine", "true") \
    .option("encoding", "SJIS").csv("/Users/yuki/pyspark_batch/dataset/jinko.csv", header=False, sep=',', inferSchema=False,schema=struct)

# 変換を行う(集計等を行う)

In [5]:
# ここでDataFrameの処理と違いSQLの場合は、仮想的なテーブルjinkoを作成します
# テーブルを作成することでSQLを発行することができるようになります
df.createOrReplaceTempView("jinko")

In [4]:
# 早速SQLを実行してみます
spark.sql("select * from jinko").show()

+--------------+----------+-----+----------+----------+----+------------+----------+------------+
|          code|    kenmei|gengo|    wareki|   seireki| chu|       sokei|jinko_male|jinko_female|
+--------------+----------+-----+----------+----------+----+------------+----------+------------+
|都道府県コード|都道府県名| 元号|和暦（年）|西暦（年）|  注|人口（総数）|人口（男）|  人口（女）|
|            00|      全国| 大正|         9|      1920|null|    55963053|  28044185|    27918868|
|            01|    北海道| 大正|         9|      1920|null|     2359183|   1244322|     1114861|
|            02|    青森県| 大正|         9|      1920|null|      756454|    381293|      375161|
|            03|    岩手県| 大正|         9|      1920|null|      845540|    421069|      424471|
|            04|    宮城県| 大正|         9|      1920|null|      961768|    485309|      476459|
|            05|    秋田県| 大正|         9|      1920|null|      898537|    453682|      444855|
|            06|    山形県| 大正|         9|      1920|null|      968925|    478328|      49059

# 前のチャプターでDataframeベースで実行していた処理をSparkSQLを使って実装し直してみます。

```
df_after_t=df.where(df.gengo == "平成").groupBy("kenmei") \
    .agg(sf.avg("jinko_male").alias("male_avg"),sf.avg("jinko_female").alias("female_avg")) \
      .filter(df.kenmei != "人口集中地区以外の地区").sort("male_avg")
```

In [5]:
# SQLで書き直すと、等価な処理は以下になります。

df_after_t=spark.sql(""" 

select kenmei,avg(jinko_male) as male_avg,avg(jinko_female) as female_avg 
 from jinko
  where gengo='平成' and kenmei != '人口集中地区以外の地区'
 group by kenmei
 order by male_avg

""")

df_after_t.show()


+--------+-----------------+-----------------+
|  kenmei|         male_avg|       female_avg|
+--------+-----------------+-----------------+
|  鳥取県|287885.3333333333|314291.3333333333|
|  島根県|         356034.5|388621.6666666667|
|  高知県|372268.1666666667|         418517.0|
|  徳島県|383399.1666666667|         423152.0|
|  福井県|         395512.5|420182.6666666667|
|  佐賀県|         408192.5|         456442.5|
|  山梨県|425777.8333333333|441831.1666666667|
|  香川県|485871.8333333333|523763.6666666667|
|和歌山県|         490624.0|547112.3333333334|
|  富山県|         532857.0|573049.8333333334|
|  宮崎県|         542386.5|         608793.0|
|  秋田県|542928.3333333334|         604578.5|
|  石川県|         566064.0|         604518.5|
|  大分県|571530.6666666666|638773.6666666666|
|  山形県|         583603.5|627811.6666666666|
|  沖縄県|         654622.0|679050.6666666666|
|  岩手県|659592.6666666666|714973.1666666666|
|  滋賀県|         662391.0|         680326.0|
|  奈良県|671178.6666666666|734736.6666666666|
|  青森県|675238.6666666666

# テーブル定義はどこに保存されている？

1. テンポラリーテーブル（メモリ）
2. Create External TABLE（メタデータストア）
   1. ローカルの場合metastore_db
   2. 本番環境などの場合は、自前で構築する場合はMysql。クラウドサービスであればAmazon Glue Data Catalogなどがあります。

今回はローカル環境なのでmetastore_dbと呼ばれるところに格納されています。
永続的に保存されるので、前のチャプターで以下のデータベースを作っている人は定義が残っているはずです。

In [ ]:
#　テーブルの作成
spark.sql(""" 
CREATE EXTERNAL TABLE IF NOT EXISTS default.jinko_avg ( male_avg double, female_avg double)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
LOCATION '/Users/yuki/pyspark_batch/dataset/parquet/';
""")

# Insert 文の発行
SparkSQLでは Insert文も発行することが可能です  
dataframeを吐き出してパーティションを認識させて。。というのは少し面倒  
insert 文を使えばその作業を簡略化できます。

# データの登録を行います

次に紹介するspark sqlを用いたinsert(とselect)は前セクションで紹介した以下のコード実行と等価です。

```
df_after_t=df.where(df.gengo == "平成").groupBy("kenmei") \
    .agg(sf.avg("jinko_male").alias("male_avg"),sf.avg("jinko_female").alias("female_avg")) \
      .filter(df.kenmei != "人口集中地区以外の地区").sort("male_avg")
```

```
df_after_t.repartition(1).write.partitionBy("kenmei").mode("overwrite").parquet("/Users/yuki/pyspark_batch/dataset/parquet")
```

```
spark.sql("msck repair table jinko_avg")

```


In [6]:
# パーティションであるkenmeiは一番うしろに記載する。
# dynamic partitionと呼ばれる機能で、kenmeiごとにパーティションを振り分けてくれます
dataframe=spark.sql(""" 
Insert overwrite table jinko_avg PARTITION(kenmei)

select avg(jinko_male) as male_avg,avg(jinko_female),kenmei as female_avg 
 from jinko
  where gengo='平成' and kenmei != '人口集中地区以外の地区'
 group by kenmei
 order by male_avg

""")

21/10/12 11:01:02 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
21/10/12 11:01:02 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
21/10/12 11:01:02 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/10/12 11:01:02 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
21/10/12 11:01:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
21/10/12 11:01:13 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


In [7]:
spark.sql("select * from jinko_avg").show()

+--------------------+--------------------+------------+
|            male_avg|          female_avg|      kenmei|
+--------------------+--------------------+------------+
|            893167.5|   944959.6666666666|      三重県|
|  1268325.3333333333|  1360099.3333333333|      京都府|
|4.0840519833333336E7|4.2415789666666664E7|人口集中地区|
|            408192.5|            456442.5|      佐賀県|
|         6.1816723E7|6.4687834833333336E7|        全国|
|           2650310.5|           2861527.0|      兵庫県|
|  2665781.3333333335|  2923371.8333333335|      北海道|
|           2987847.0|           2974638.5|      千葉県|
|            490624.0|   547112.3333333334|    和歌山県|
|  3492880.3333333335|  3443447.8333333335|      埼玉県|
|   571530.6666666666|   638773.6666666666|      大分県|
|   4292675.833333333|           4517115.0|      大阪府|
|   671178.6666666666|   734736.6666666666|      奈良県|
|           1139561.5|           1191255.0|      宮城県|
|            542386.5|            608793.0|      宮崎県|
|            532857.0|

In [9]:
# 出来上がったファイル一度みてみましょう(ロケーションは「/Users/yuki/pyspark_batch/dataset/parquet」なのでロケーションはいかにデータが出ているはずです)
!ls -l /Users/yuki/pyspark_batch/dataset/parquet/kenmei=三重県

total 8
-rw-r--r--  1 yuki  staff  741 Oct 12 11:01 part-00025-7388977f-70b8-4baa-8470-a14ee29657b0.c000.snappy.parquet


# hint文
SQLを実行するときでもスモールファイル問題に対応できるようにrepartitionに相当する機能が存在しています。

それがヒント文です。

```
/** REPARTITION(25) */

```

Dataframeの操作では以下の部分です。  
df_after_t.repartition(1)

先程のinsert文は実はそのままでも動くのですが、スモールファイル問題を引き起こす原因にもなってしまいます。  
そこでこのヒント文を使って書き直してみます。

In [23]:
# ヒント文をSelectの部分に埋め込無事でファイルのばらつきを抑える(数字は２とすれば、パーティションごとに２つのファイルができるYewYe)
dataframe=spark.sql(""" 
Insert overwrite table jinko_avg PARTITION(kenmei)

select /** REPARTITION(10) */ avg(jinko_male) as male_avg,avg(jinko_female),kenmei as female_avg 
 from jinko
  where gengo='平成' and kenmei != '人口集中地区以外の地区'
 group by kenmei
 order by male_avg

""")

In [24]:
# 出来上がったファイル一度みてみましょう(ロケーションは「/Users/yuki/pyspark_batch/dataset/parquet」なのでロケーションはいかにデータが出ているはずです)
!ls -l /Users/yuki/pyspark_batch/dataset/parquet/kenmei=三重県

total 8
-rw-r--r--  1 yuki  staff  741 Oct 12 11:14 part-00025-7f783241-3290-4831-b9c3-bdf08ce5d22d.c000.snappy.parquet


# update文やdelete文は？
Update/delete文はありません。  
ビッグデータの世界では、トランザクションシステムのように対象のレコードだけをピンポイントで更新する機能を有していないものが多いです。  
Sparkもその考えからUpdate/delete文を持ち合わせていません。　　

何兆というレコードから対象のレコードを探すことはかなり難しいのと、たくさんのデータソースがまじり合うデータ基盤では
トランザクションシステムのように、PKを設定することが困難なことが挙げられます

# SparkSQLを使うメリット

- SQLを使った処理はSQLファイルを外部に配置してそのファイルを読み込み実行することで汎用化がしやすい
- SQLになれた人であれば操作がしやすい
- データエンジニアリングとしては、SparkSQLを使ってシステム構築をするほうが汎用的で容易

# DataFrameを使うメリット
- データフレームにしかできないような仕事もある。例えば行列の変換などはdataframeのほうが実行しやすいのでデータサイエンスを好む人はDataFrameを使うほうが良い場合もある。

## 速度は変わる？
速度はどちらでも変わりません。内部的にはSQLはDataFrameとして処理され実行されます。

In [ ]:
# Spark利用の停止
spark.stop()
spark.sparkContext.stop()